In [688]:
import numpy as np
import matplotlib.pyplot as plt
from typing import Callable, Optional, List, Set

# Project Idea: Find the (Non-Complex) Eigenvalue of a Matrix

It's from scratch-ish with some use of `numpy` because I don't want to write a hundred for-loops.

## Part 1: Finding the determinant of a matrix

The determinant is very useful for so many things related to matrix algebra. One of its applications is in finding the eigenvalues of a matrix.

If I'm not lazy, I'll come back to write the explanation of how to find the determinant of a matrix. In the meanwhile, have [this](https://www.mathsisfun.com/algebra/matrix-determinant.html).



In [689]:
def forward_elim(A):
	L = A.copy()

	n_row, _ = L.shape
	for p in range(n_row):
		for j in range(p+1, n_row):
			pivot = L[p, p]
			coeff = L[j, p] / pivot
			L[j] = L[j] - coeff * L[p]
			# print(L, end="\n\n")

	return L

In [690]:
"""
Find the determinant of a given matrix (recursively)

Param: A, input matrix
Returns: determinant of A
"""
def det(A:np.ndarray) -> float:
	A = A.astype(float)
	size, size2 = A.shape
	if size != size2:
		raise ValueError("Matrix A must be square")

	# if size == 2:
	# 	return A[0, 0]*A[1, 1] - A[1, 0]*A[0, 1]

	# else:
	# 	accumulator = 0
	# 	sign = -1
	# 	for i in range(size):
	# 		sub_matrix = np.array([ [ A[row, col] for col in range(size) if col != i ] for row in range(1, size) ])
	# 		accumulator += (sign**i) * A[0, i]*det(sub_matrix)
	# 	return accumulator

	reduced = forward_elim(A)
	d = reduced[0, 0]
	for i in range(1, size):
		d *= reduced[i, i]

	return d

In [691]:
np.random.seed(123)
test_matrix = np.random.randint(1, 10, size=(2, 2))
print("np det:", np.linalg.det(test_matrix))
print("my det:", det(test_matrix))

test_matrix = np.random.randint(1, 20, size=(3, 3))
# print(test_matrix)
print("np det:", np.linalg.det(test_matrix))
print("my det:", det(test_matrix))

np det: -15.0
my det: -15.0
np det: -1235.9999999999995
my det: -1236.0


## Part 2: Fancy root finding

In Linear Algebra B, we learned that we can find the eigenvalues of a matrix by finding the roots or values of $\lambda$ such that $\det(A - \lambda I) = 0$. 

To find the roots, we will use Newton's method because it's faster so it should give us slightly better performance since we might need to find multiple roots.

In [692]:
"""
Find the derivative of a given point of function f using the centering method

Params:
* f: function to be used
* x: given point to find the derivative

Returns: slope at given point
"""
def derivative(f:Callable[[float], float], x:float):
	h = 0.01
	return (f(x+h) - f(x-h)) / 2/h

In [693]:
"""
Fancier version of Newton's method

Params:
* f: the function such that we want to find the point that f(x) = 0
* l: initial left bound
* r: initial right bound

Returns: a root in given bound rounded to 6 digits or None if there's no root or takes too long
"""
def newton(f:Callable[[float], float], x0) -> Optional[float]:
	BAD_TIME = 30
	x = x0
	c = 0
	while f(x) != 0 and c < BAD_TIME:
		x = x - (f(x) / derivative(f, x))
		c += 1
	
	if f(x) == 0:
		return round(x, 5)
	elif abs(f(x)) < 1e-10:
		return round(x, 5)
	else:
		return None

In [694]:
# newton(lambda x : np.cos(x) - x, 0)

In [695]:
"""
For finding multiple roots within the given bound, using the shotgunning along with Newton's method

Params:
* f: the function we want to find roots for f(x) = 0
* l: left bound
* r: right bound
* n: how many initial points do we want

Returns: list of rounded roots if any
"""
def find_roots(f:Callable[[float], float], l:float, r:float) -> Set[float]:
	n = 100
	roots = set()
	for i in range(int(n)):
		try:
			res = newton(f, l + d*i)
		except ZeroDivisionError: # catch the local max/min
			res = None
		if res != None:
			roots.add(res)
	return roots

In [696]:
# find_roots(lambda x: x**2+x-5, -100, 100)

## Part 3: Finding the eigenvalues

This is the first difficult part.

While it is quite simple to find the values of $\lambda$ such that it satisfies the condition $\det(A - \lambda I) = 0$,
it is a challenge on how to choose the initial values for Newton's method for this.

However, since we know that the product of all eigenvalues is the determinant,
we can use the different values between the negative and positive values of the determinant to do the root finding.

In [697]:
"""
Creating the function to calculate the det(A-lmbda*I) = 0

Param: matrix A
Returns: function that calculates det(A-lambda*I) for given lambda
"""
def zero_det(A:np.ndarray) -> Callable[[float], float]:
	def func(lmbda:float) -> float:
		size, _ = A.shape
		return det(A - lmbda*np.identity(size))
	return func

In [698]:
# taken from Linear B notes
test_matrix = np.array([[5., 8., 16.],
						[4., 1., 8.],
						[-4., -4., -11.]])
d = det(test_matrix)
print("matrix:")
print(test_matrix)
print("det:", d)
find_roots(zero_det(test_matrix), -abs(d), abs(d))

matrix:
[[  5.   8.  16.]
 [  4.   1.   8.]
 [ -4.  -4. -11.]]
det: 8.999999999999993


{-3.0, 1.0}

Finally, let us combine everything we've written above into a function to find the eigenvalues of given matrix.

In [699]:
def find_eigvals(A:np.ndarray) -> Set[float]:
	d = det(A)
	return find_roots(zero_det(A), -abs(d), abs(d))

In [700]:
print("home-baked:\t", 	find_eigvals(test_matrix)) # less accurate, no complex roots, but home-baked
print("numpy:\t\t", 	np.linalg.eigvals(test_matrix)) # better in every way but it's by numpy

home-baked:	 {1.0, -3.0}
numpy:		 [ 1.+0.00000000e+00j -3.+2.71947991e-15j -3.-2.71947991e-15j]
